In [1]:
!pip install transformers==4.13.0
!pip install datasets==2.0.0
!pip install accelerate==0.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 7.3 MB/s 
     |████████████████████████████████| 163 kB 28.3 MB/s 
     |████████████████████████████████| 3.3 MB 42.6 MB/s 
     |████████████████████████████████| 880 kB 55.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=4389ea16ddcbc893505715dd4aa0f6ad55b4c24db77908775b61aed32b30a618
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 325 kB 8.0 MB/s 
     |████████████████████████████████| 115 kB 56.5 MB/s 
     |████████████████████████████████| 212 kB 63.8 MB/s 
     |████████████████████████████████| 127 kB 56.6 MB/s 
  Attempting uninstall: urllib3
    Found e

# Custom Named Entity Recognition(NER)

## What is NER?
Named-entity recognition (NER) (also known as (named) entity identification, entity chunking, and entity extraction) is a subtask of information extraction that seeks to locate and classify *named entities* mentioned in unstructured text into pre-defined categories such as person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc.

`named Entity` -: It is a real-world object, such as a person, location, organization, product, etc.

## What is the Dataset format for Name Entity Recognition(NER)?

Approaches typically use BIO notation, which differentiates the beginning (B) and the inside (I) of entities. O is used for non-entity tokens. 

Various Benchmark Datasets available for training and evaluation NER models are-:
- [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003)
- [CrossNER](https://paperswithcode.com/dataset/crossner)
- [XTREME](https://arxiv.org/abs/2003.11080)

In [2]:
#id jeff-dean-ner
#caption An example of a sequence annotated with named entities
#hide_input
import pandas as pd
toks = "Jeff Dean is a computer scientist at Google Rishav facebook Snapchat in California".split()
lbls = ["B-PER", "I-PER", "O", "O", "O", "O", "O", "B-ORG", "B-PER","B-ORG", "I-ORG", "O", "B-LOC"]
df = pd.DataFrame(data=[toks, lbls], index=['Tokens', 'Tags'])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
Tokens,Jeff,Dean,is,a,computer,scientist,at,Google,Rishav,facebook,Snapchat,in,California
Tags,B-PER,I-PER,O,O,O,O,O,B-ORG,B-PER,B-ORG,I-ORG,O,B-LOC


## Dataset to be used

Here we will be using a subset of the **Cross-lingual TRansfer Evaluation of Multilingual Encoders (XTREME)** benchmark called **WikiANN** or **PAN-X.2** This dataset consists of Wikipedia articles in many languages, including the four most commonly spoken languages in Switzerland: German (62.9%), French (22.9%), Italian (8.4%), and English (5.9%). Each article is annotated with LOC (location), PER (person), and ORG (organization) tags in the ***“inside-outside-beginning” (IOB2)*** format.

## Get the Dataset configuration 

There are many configuration available for the XTREME dataset. We have to select only the required one.

In [3]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME has {len(xtreme_subsets)} configurations")

XTREME has 183 configurations


Whoa, that’s a lot of configurations! Let’s narrow the search by just looking for the configurations that start with “PAN”

In [4]:
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
panx_subsets[:3]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

In [5]:
panx_subsets

['PAN-X.af',
 'PAN-X.ar',
 'PAN-X.bg',
 'PAN-X.bn',
 'PAN-X.de',
 'PAN-X.el',
 'PAN-X.en',
 'PAN-X.es',
 'PAN-X.et',
 'PAN-X.eu',
 'PAN-X.fa',
 'PAN-X.fi',
 'PAN-X.fr',
 'PAN-X.he',
 'PAN-X.hi',
 'PAN-X.hu',
 'PAN-X.id',
 'PAN-X.it',
 'PAN-X.ja',
 'PAN-X.jv',
 'PAN-X.ka',
 'PAN-X.kk',
 'PAN-X.ko',
 'PAN-X.ml',
 'PAN-X.mr',
 'PAN-X.ms',
 'PAN-X.my',
 'PAN-X.nl',
 'PAN-X.pt',
 'PAN-X.ru',
 'PAN-X.sw',
 'PAN-X.ta',
 'PAN-X.te',
 'PAN-X.th',
 'PAN-X.tl',
 'PAN-X.tr',
 'PAN-X.ur',
 'PAN-X.vi',
 'PAN-X.yo',
 'PAN-X.zh']

OK, it seems we’ve identified the syntax of the PAN-X subsets: each one has a two-letter suffix that appears to be an ISO 639-1 language code. This means that to load the English corpus, we pass the en code to the name argument of load_dataset() as follows:

In [6]:
# hide_output
from datasets import load_dataset

load_dataset("xtreme", name="PAN-X.en")

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset xtreme downloaded and prepared to /root/.cache/huggingface/datasets/xtreme/PAN-X.en/1.0.0/2fc6b63c5326cc0d1f73060649612889b3a7ed8a6605c91cecdbd228a7158b17. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
})

Loading the dataset to a variable

In [7]:
en = load_dataset("xtreme", name='PAN-X.en')

  0%|          | 0/3 [00:00<?, ?it/s]

Let's check what all keys we are having for this dataset"

In [8]:
en.keys()

dict_keys(['validation', 'test', 'train'])

Now let's access the training dataset and check what all data we have in each individual index

In [9]:
en_train = en['train']

We get the 14th element of the training dataset. There are 3 keys available in it. 

The `tokens` consists of word tokenized format of a sentence. The `ner_tag` is having the index value of different tags. `langs` consist the language each token is referring, here it's english.

In [10]:
en_train[14]

{'tokens': ['James', 'Franco', '(', 'M.F.A', '.'],
 'ner_tags': [1, 2, 0, 0, 0],
 'langs': ['en', 'en', 'en', 'en', 'en']}

Creating a Dataframe for better representation.

In [11]:
pd.DataFrame(en["train"][0]).transpose()

,0,1,2,3,4,5,6,7,8,9,10
tokens,R.H.,Saunders,(,St.,Lawrence,River,),(,968,MW,)
ner_tags,3,4,0,3,4,4,0,0,0,0,0
langs,en,en,en,en,en,en,en,en,en,en,en


This is a bit cryptic to the human eye, so let’s create a new column with the familiar `LOC`, `PER`, and `ORG` tags. To do this, the first thing to notice is that our Dataset object has a features attribute that specifies the underlying data types associated with each column:

In [12]:
for key, value in en["train"].features.items():
    print(f"{key}: {value} \n")

tokens: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None) 

ner_tags: Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None) 

langs: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None) 



The `Sequence` class specifies that the field contains a list of features, which in the case of `ner_tags` corresponds to a list of `ClassLabel` features. Let’s pick out this feature from the training set as follows:

In [13]:
tags = en["train"].features["ner_tags"].feature
print(tags)

ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)


We can use the `ClassLabel.int2str()` method to create a new column in our training set with class names for each tag.
We’ll use the `map()` method to return a `dict` with the key corresponding to the new column name and the value as a `list` of class names:

In [14]:
# hide_output
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}

panx_en = en.map(create_tag_names)

  0%|          | 0/10000 [00:00<?, ?ex/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

  0%|          | 0/20000 [00:00<?, ?ex/s]

Now that we have our tags in human-readable format, let’s see how the tokens and tags align for the first example in the training set:

In [15]:
# hide_output
de_example = panx_en["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]],
['Tokens', 'Tags'])

,0,1,2,3,4,5,6,7,8,9,10
Tokens,R.H.,Saunders,(,St.,Lawrence,River,),(,968,MW,)
Tags,B-ORG,I-ORG,O,B-ORG,I-ORG,I-ORG,O,O,O,O,O


As a quick check that we don’t have any unusual imbalance in the tags, let’s calculate the frequencies of each entity across each split:

This looks good—the distributions of the `PER`, `LOC`, and `ORG` frequencies are roughly the same for each split, so the validation and test sets should provide a good measure of our NER tagger’s ability to generalize.

In [16]:
from collections import Counter
from collections import defaultdict
from datasets import DatasetDict
split2freqs = defaultdict(Counter)
for split, dataset in panx_en.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs, orient="index")

,ORG,LOC,PER
validation,4677,4834,4635
test,4745,4657,4556
train,9422,9345,9164


## Model we would be using 


- One of the first multilingual transformers was [mBERT](https://github.com/google-research/bert/blob/master/multilingual.md), which uses the same architecture and pretraining objective as BERT but adds Wikipedia articles from many languages to the pretraining corpus. 

- Since then, mBERT has been superseded by [XLM-RoBERTa](https://arxiv.org/abs/1911.02116) (or XLM-R for short), so that’s the model we’ll consider in this chapter.

- XLM-R uses only MLM as a pretraining objective for 100 languages, but is distinguished by the huge size of its pretraining corpus compared to its predecessors: Wikipedia dumps for each language and 2.5 terabytes of Common Crawl data from the web. 

- This corpus is several orders of magnitude larger than the ones used in earlier models and provides a significant boost in signal for low-resource languages like Burmese and Swahili, where only a small number of Wikipedia articles exist.

- The RoBERTa part of the model’s name refers to the fact that the pretraining approach is the same as for the monolingual RoBERTa models. RoBERTa’s developers improved on several aspects of BERT, in particular by removing the next sentence prediction task altogether.3 XLM-R also drops the language embeddings used in XLM and uses SentencePiece to tokenize the raw texts directly.4 Besides its multilingual nature, a notable difference between XLM-R and RoBERTa is the size of the respective vocabularies: 250,000 tokens versus 55,000!


## Tokenization
Simply tokenization is a operation which transforms a string to integer that can be passed through the model.

The steps included in Tokenization are-:
1. Normailization.
2. Pretokenization.
3. Tokenizer Model.
4. Postprocessing.

### 1. Normailization
- This step corresponds to the set of operations you apply to a raw string to make it “cleaner.”
- Some of the common operations include stripping whitespaces, removing accented characters, Unicode Normalization, lowercasing. 

### 2. Pretokenization
-  This step splits a text into smaller objects that give an upper bound to what your tokens will be at the end of training.
- This basically splits the text into "words", and the final tokens will be parts of those words.
- Strings can typically be split into words on whitespace and punctuation.

### 3. Tokenizer model
- Once the input texts are normalized and pretokenized, the tokenizer applies a subword splitting model on the words.
- This is the part of the pipeline that needs to be trained on your corpus(or that has been trained if you are using a pretrained tokenizer).
- The role of the model is to split the words into subwords to reduce the size of the vocabulary and try to reduce the number of out-of-vocabulary tokens. Several subword tokenization algorithms exist, including BPE, Unigram, and WordPiece.
- For instance, our running example might look like `[jack, spa, rrow, loves, new, york, !]` after converting it from `["jack", "sparrow", "loves", "new", "york", "!"]`.
-  Note that at this point we no longer have a list of strings but a list of integers (input IDs); to keep the example illustrative, we’ve kept the words but dropped the quotes to indicate the transformation.

### 4. Postprocessing
- This step include some additional transformation for instance, adding special tokens at the beginning or end of the input sequence of token indices. For example, a BERT-style tokenizer would add classifications and separator tokens: `[CLS, jack, spa, rrow, loves, new, york, !, SEP]`.

- This sequence can be then fed to the model.


As XLM-R uses SentencePiece tokenizer as we saw most places WordPiece tokenizers are used. Let's compare how these two tokenizers are different.


For that we would be using HuggingFace AutoTokenizer and pretrained model of both `bert-base-cased` and `xlm-roberta-base`.


In [17]:
# hide_output
from transformers import AutoTokenizer

bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

By encoding a small sequence of text we can also retrieve the special tokens that each model used during pretraining:

In [18]:
text = "Jack Sparrow loves New York!"
bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()

Here we see that instead of the `[CLS]` and `[SEP]` tokens that BERT uses for sentence classification tasks, XLM-R uses `<s>` and `<\s>` to denote the start and end of a sequence.

In [19]:
#hide_input
df = pd.DataFrame([bert_tokens, xlmr_tokens], index=["BERT", "XLM-R"])
df

,0,1,2,3,4,5,6,7,8,9
BERT,[CLS],Jack,Spa,##rrow,loves,New,York,!,[SEP],None
XLM-R,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>


### The SentencePiece Tokenizer

- The SentencePiece tokenizer is based on a type of subword segmentation called Unigram and encodes each input text as a sequence of Unicode characters.

-  This last feature is especially useful for multilingual corpora since it allows SentencePiece to be agnostic about accents, punctuation, and the fact that many languages, like Japanese, do not have whitespace characters.

- Another special feature of SentencePiece is that whitespace is assigned the Unicode symbol U+2581, or the ▁ character, also called the lower one quarter block character.

-  This enables SentencePiece to detokenize a sequence without ambiguities and without relying on language-specific pretokenizers.

- Here if we see that WordPiece has lost the information that there is no whitespace between “York” and “!”.

- By contrast, SentencePiece preserves the whitespace in the tokenized text so we can convert back to the raw text without ambiguity:

In [20]:
"".join(xlmr_tokens).replace(u"\u2581", " ")

'<s> Jack Sparrow loves New York!</s>'

## Transformers for Named Entity Recognition

We know that for text classification BERT uses the special [CLS] token to represent an entire sequence of text. This representation is then fed through a fully connected or dense layer to output the distribution of all the discrete label values, as shown:

<img alt="Architecture of a transformer encoder for classification." caption="Fine-tuning an encoder-based transformer for sequence classification" src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter04_clf-architecture.png?raw=1" id="clf-arch"/>

BERT and other encoder-only transformers take a similar approach for NER, except that the representation of each individual input token is fed into the same fully connected layer to output the entity of the token. For this reason, NER is often framed as a `token classification task`.


Only difference is the assignment of label to the subword ("##port"). Here we will assign the subword with `IGN` label. This can be later handled in the postprocessing step. 













<img alt="Architecture of a transformer encoder for named entity recognition. The wide linear layer shows that the same linear layer is applied to all hidden states." caption="Fine-tuning an encoder-based transformer for named entity recognition" src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter04_ner-architecture.png?raw=1" id="ner-arch"/>

## Transfer Learning using Hugging Face Transformers

### Bodies and Heads

Model head has the last layer which is task specific. The rest of the model is called the body; it includes the token embeddings and transformer layers that are task-agnostic. 


<img alt="bert-body-head" caption="The `BertModel` class only contains the body of the model, while the `BertFor&lt;Task&gt;` classes combine the body with a dedicated head for a given task" src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter04_bert-body-head.png?raw=1" id="bert-body-head"/>

### Creating a Custom Model for Token Classification

This is just an example of a custom model for Token classification, already we have `XLMRobertaForTokenClassification ` in huggingface which can be directly used.



In [21]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        # Load model body
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        # Set up token classification head
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # Load and initialize weights
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, 
                labels=None, **kwargs):
        # Use model body to get encoder representations
        outputs = self.roberta(input_ids, attention_mask=attention_mask,
                               token_type_ids=token_type_ids, **kwargs)
        # Apply classifier to encoder representation
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        # Calculate losses
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        # Return model output object
        return TokenClassifierOutput(loss=loss, logits=logits, 
                                     hidden_states=outputs.hidden_states, 
                                     attentions=outputs.attentions)

### Loading a Custom Model

- Now we are ready to load our token classification model. We’ll need to provide some additional information beyond the model name, including the tags that we will use to label each entity and the mapping of each tag to an ID and vice versa. 
- All of this information can be derived from our tags variable, which as a ClassLabel object has a names attribute that we can use to derive the mapping:

In [22]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

In [23]:
tag2index

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6}

Now, we can load the model weights as usual with the from_pretrained() function with the additional config argument. Note that we did not implement loading pretrained weights in our custom model class; we get this for free by inheriting from RobertaPreTrainedModel


The `AutoConfig` class contains the blueprint of a model’s architecture. When we load a model with `AutoModel.from_pretrained(model_ckpt)`, the configuration file associated with that model is downloaded automatically. However, if we want to modify something like the number of classes or label names, then we can load the configuration first with the parameters we would like to customize.

In [24]:
# hide_output
from transformers import AutoConfig
xlmr_model_name = "xlm-roberta-base"
xlmr_config = AutoConfig.from_pretrained(xlmr_model_name, 
                                         num_labels=tags.num_classes,
                                         id2label=index2tag, label2id=tag2index)

In [ ]:
xlmr_config

Now, we can load the model weights as usual with the `from_pretrained()`function with the additional `config` argument. Note that we did not implement loading pretrained weights in our custom model class; we get this for free by inheriting from `RobertaPreTrainedModel`:

In [26]:
# hide_output
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xlmr_model = (XLMRobertaForTokenClassification
              .from_pretrained(xlmr_model_name, config=xlmr_config)
              .to(device))

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'roberta

In [27]:
text

'Jack Sparrow loves New York!'

Let's check the tokenizer on small texzt as Example

In [28]:
# hide_output
input_ids = xlmr_tokenizer.encode(text, return_tensors="pt")
pd.DataFrame([xlmr_tokens, input_ids[0].numpy()], index=["Tokens", "Input IDs"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Input IDs,0,21763,37456,15555,5161,7,2356,5753,38,2


Passing the tokens to the `XLMR` model.

Here we see that the logits have the shape `[batch_size, num_tokens, num_tags]`, with each token given a logit among the seven possible NER tags.



In [36]:
outputs = xlmr_model(input_ids.to(device)).logits
predictions = torch.argmax(outputs, dim=-1)
print(f"Number of tokens in sequence: {len(xlmr_tokens)}")
print(f"Shape of outputs: {outputs.shape}")

Number of tokens in sequence: 10
Shape of outputs: torch.Size([1, 10, 7])


In [30]:
preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens, preds], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Tags,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC


Accumulating all the steps in opne functions

In [31]:
def tag_text(text, tags, model, tokenizer):
    # Get tokens with special characters
    tokens = tokenizer(text).tokens()
    # Encode the sequence into IDs
    input_ids = xlmr_tokenizer(text, return_tensors="pt").input_ids.to(device)
    # Get predictions as distribution over 7 possible classes
    outputs = model(input_ids)[0]
    # Take argmax to get most likely class per token
    predictions = torch.argmax(outputs, dim=2)
    # Convert to DataFrame
    preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
    return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])
    

## Tokenizing Texts for NER

In [32]:
de_example

{'tokens': ['R.H.',
  'Saunders',
  '(',
  'St.',
  'Lawrence',
  'River',
  ')',
  '(',
  '968',
  'MW',
  ')'],
 'ner_tags': [3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0],
 'langs': ['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en'],
 'ner_tags_str': ['B-ORG',
  'I-ORG',
  'O',
  'B-ORG',
  'I-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'O',
  'O']}

In [39]:
words, labels = de_example["tokens"], de_example["ner_tags"]

In [40]:
tokenized_input = xlmr_tokenizer(de_example["tokens"], is_split_into_words=True)
tokens = xlmr_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])                                    

In [38]:
tokenized_input

{'input_ids': [0, 627, 5, 841, 5, 947, 24658, 7, 15, 2907, 5, 155484, 32547, 1388, 15, 483, 16028, 46298, 1388, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [41]:
tokens

['<s>',
 '▁R',
 '.',
 'H',
 '.',
 '▁Sa',
 'under',
 's',
 '▁(',
 '▁St',
 '.',
 '▁Lawrence',
 '▁River',
 '▁)',
 '▁(',
 '▁9',
 '68',
 '▁MW',
 '▁)',
 '</s>']

In [35]:
#hide_output
pd.DataFrame([tokens], index=["Tokens"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,<s>,▁R,.,H,.,▁Sa,under,s,▁(,▁St,.,▁Lawrence,▁River,▁),▁(,▁9,68,▁MW,▁),</s>


In [37]:
# hide_output
word_ids = tokenized_input.word_ids()
pd.DataFrame([tokens, word_ids], index=["Tokens", "Word IDs"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,<s>,▁R,.,H,.,▁Sa,under,s,▁(,▁St,.,▁Lawrence,▁River,▁),▁(,▁9,68,▁MW,▁),</s>
Word IDs,None,0,0,0,0,1,1,1,2,3,3,4,5,6,7,8,8,9,10,None


Here we can see that `word_ids` has mapped each subword to the corresponding index in the `words` sequence, so the first subword, “▁2.000”, is assigned the index 0, while “▁Einwohner” and “n” are assigned the index 1 (since “Einwohnern” is the second word in `words`). We can also see that special tokens like `<s>` and `<\s>` are mapped to `None`. Let’s set –100 as the label for these special tokens and the subwords we wish to mask during training:

In [ ]:
#hide_output
previous_word_idx = None
label_ids = []

for word_idx in word_ids:
    if word_idx is None or word_idx == previous_word_idx:
        label_ids.append(-100)
    elif word_idx != previous_word_idx:
        label_ids.append(labels[word_idx])
    previous_word_idx = word_idx
    
labels = [index2tag[l] if l != -100 else "IGN" for l in label_ids]
index = ["Tokens", "Word IDs", "Label IDs", "Labels"]

pd.DataFrame([tokens, word_ids, label_ids, labels], index=index)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,<s>,▁R,.,H,.,▁Sa,under,s,▁(,▁St,.,▁Lawrence,▁River,▁),▁(,▁9,68,▁MW,▁),</s>
Word IDs,None,0,0,0,0,1,1,1,2,3,3,4,5,6,7,8,8,9,10,None
Label IDs,-100,3,-100,-100,-100,4,-100,-100,0,3,-100,4,4,0,0,0,-100,0,0,-100
Labels,IGN,B-ORG,IGN,IGN,IGN,I-ORG,IGN,IGN,O,B-ORG,IGN,I-ORG,I-ORG,O,O,O,IGN,O,O,IGN


### Why did we choose –100 as the ID to mask subword representations?

The reason is that in PyTorch the cross-entropy loss class torch.nn.CrossEntropyLoss has an attribute called ignore_index whose value is –100. This index is ignored during training, so we can use it to ignore the tokens associated with consecutive subwords.


Accumulating all the steps in a functions:

In [42]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(examples["tokens"], truncation=True, 
                                      is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Now we have all the functions. We can map it on whole dataset that we had defined above.

In [43]:
def encode_panx_dataset(corpus):
    return corpus.map(tokenize_and_align_labels, batched=True, 
                      remove_columns=['langs', 'ner_tags', 'tokens'])

In [44]:
# hide_output
panx_de_encoded = encode_panx_dataset(panx_en)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/20 [00:00<?, ?ba/s]

In [45]:
panx_de_encoded['train']

Dataset({
    features: ['ner_tags_str', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 20000
})

## Performance Measures


- Evaluating a NER model is similar to evaluating a text classification model, and it is common to report results for precision, recall, and F1-score.
- The only subtlety is that all words of an entity need to be predicted correctly in order for a prediction to be counted as correct. 
- Fortunately, there is a nifty library called seqeval that is designed for these kinds of tasks. 

In [46]:
!pip install seqeval==1.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.7 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=b56e1a17a66a3f3b7b0a776ec1e560df4026f97f3089c36242d167d7e4bda0a4
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [47]:
from seqeval.metrics import classification_report

y_true = [["O", "O", "O", "B-MISC", "I-MISC", "I-MISC", "O"],
          ["B-PER", "I-PER", "O"]]
y_pred = [["O", "O", "B-MISC", "I-MISC", "I-MISC", "I-MISC", "O"],
          ["B-PER", "I-PER", "O"]]
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

        MISC       0.00      0.00      0.00         1
         PER       1.00      1.00      1.00         1

   micro avg       0.50      0.50      0.50         2
   macro avg       0.50      0.50      0.50         2
weighted avg       0.50      0.50      0.50         2



In [ ]:
import numpy as np

def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # Ignore label IDs = -100
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

## Fine-Tuning XLM-RoBERTa

We will use the Transformers `Trainer` to handle our training loop, so first we need to define the training attributes using the `TrainingArguments` class:

In [ ]:
# hide_output
from transformers import TrainingArguments

num_epochs = 3
batch_size = 24
logging_steps = len(panx_de_encoded["train"]) // batch_size
model_name = f"{xlmr_model_name}-finetuned-english"
training_args = TrainingArguments(
    output_dir=model_name, log_level="error", num_train_epochs=num_epochs, 
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=batch_size, evaluation_strategy="epoch", 
    save_steps=1e6, weight_decay=0.01, disable_tqdm=False, 
    logging_steps=logging_steps, push_to_hub=True)

In [ ]:
from seqeval.metrics import f1_score

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions, 
                                       eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred)}

The final step is to define a data collator so we can pad each input sequence to the largest sequence length in a batch.  Transformers provides a dedicated data collator for token classification that will pad the labels along with the inputs:

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

In [ ]:
def model_init():
    return (XLMRobertaForTokenClassification
            .from_pretrained(xlmr_model_name, config=xlmr_config)
            .to(device))

We can now pass all this information together with the encoded datasets to the `Trainer`:

In [ ]:
# hide_output
from transformers import Trainer

trainer = Trainer(model_init=model_init, args=training_args, 
                  data_collator=data_collator, compute_metrics=compute_metrics,
                  train_dataset=panx_de_encoded["train"],
                  eval_dataset=panx_de_encoded["validation"], 
                  tokenizer=xlmr_tokenizer)

CODECARBON : No CPU tracking mode found. Falling back on CPU constant mode.
CODECARBON : Failed to match CPU TDP constant. Falling back on a global constant.
/home/lewtun/miniconda3/envs/book/lib/python3.7/site-packages/apscheduler/util.py:95: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  if obj.zone == 'local':


### Run the training loop as follows

In [ ]:
#hide_input
trainer.train()

In [ ]:
# hide_input
df = pd.DataFrame(trainer.state.log_history)[['epoch','loss' ,'eval_loss', 'eval_f1']]
df = df.rename(columns={"epoch":"Epoch","loss": "Training Loss", "eval_loss": "Validation Loss", "eval_f1":"F1"})
df['Epoch'] = df["Epoch"].apply(lambda x: round(x))
df['Training Loss'] = df["Training Loss"].ffill()
df[['Validation Loss', 'F1']] = df[['Validation Loss', 'F1']].bfill().ffill()
df.drop_duplicates()

Epoch,Training Loss,Validation Loss,F1
1,0.2652,0.160244,0.822974
2,0.1314,0.137195,0.852747
3,0.0806,0.138774,0.864591


In [ ]:
# hide_output
text_de = "Jeff Dean ist ein Informatiker bei Google in Kalifornien"
tag_text(text_de, tags, trainer.model, xlmr_tokenizer)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
Tokens,<s>,▁Jeff,▁De,an,▁ist,▁ein,▁Informati,ker,▁bei,▁Google,▁in,▁Kaliforni,en,</s>
Tags,O,B-PER,I-PER,I-PER,O,O,O,O,O,B-ORG,O,B-LOC,I-LOC,O
